In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
from pandas.tseries.offsets import DateOffset
#from torch_geometric.data import Data

#Lets start at src location
if os.path.exists("./src"):
    os.chdir("./src")


config = {
    "counter_files_path" : "../data/counters_temporal_data_2023-03-03T09-24-06/",
    "N_GRAPHS"           : 30*24,
    "F_IN"               : 7*24,
    "F_OUT"              : 7*24,
    "target_col"         : "Sum"
}

In [7]:
class TrafficDataset:
    def __init__(self, config):
        self.config = config
        self.X = None
        self.Y = None

    def prepare_data(self):
        #First prepare general matrix X for all counters
        counters_df = pd.DataFrame()
        for fname in glob.glob(self.config["counter_files_path"] + "*.csv"):
            counter_data = pd.read_csv(fname)
            counter_data['Date'] = pd.to_datetime(counter_data['Date']) 
            counter_data.index = counter_data['Date']
            counter_data = counter_data.sort_index(ascending=False)
            # We don't need to work with all past data.
            # Select enough data points to extract N_GRAPHS with F_IN and F_OUT timepoints
            
            counter_data = counter_data.iloc[0:(self.config["F_IN"]+self.config["F_OUT"]+self.config["N_GRAPHS"]-1), :]
            counter_id = fname.split('\\')[1].split('.csv')[0]

            if counters_df.empty:
                counters_df = pd.DataFrame(counter_data[self.config['target_col']])
                counters_df.columns = [counter_id]
            else:
                columns = list(counters_df.columns) + [counter_id]
                counters_df = pd.concat([counters_df, counter_data[self.config['target_col']]], axis=1)
                counters_df.columns = columns 

        return counters_df

            

In [8]:
td = TrafficDataset(config)
df = td.prepare_data()

IndexError: list index out of range

In [ ]:
df

""
